In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

In [ ]:
import folium

In [ ]:
def cloudMask(img):
  return img.updateMask(img.select('BQA').lt(64))


def add_distance(feature):
  return feature.set({'d': feature.distance(out_ext)})
  

def select_landsat(year):
  if (year >= 2013):
    return {
        'bands': ['B2', 'B3', 'B4', 'B5', 'B6', 'B7'],
        'FCCbands': ['B5', 'B4', 'B3'],
        'TextureBand': ['B5'],
        'bandsClassify': ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','B5_1'],
        'bandsGLCM': ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B5_asm', 'B5_contrast', "B5_corr", "B5_var", "B5_idm", "B5_savg", "B5_svar", "B5_sent", "B5_ent", "B5_dvar", "B5_dent", "B5_imcorr1", "B5_imcorr2","B5_maxcorr", "B5_diss",  "B5_inertia", "B5_shade", "B5_prom"],
        'Landsat': 'LANDSAT/LC08/C01/T1'
    }

  if (year >= 1999 and year <= 2002):
    return {
        'bands': ['B1', 'B2', 'B3', 'B4', 'B5', 'B7'],
        'TextureBand': ['B4'],
        'bandsClassify': ['B1', 'B2', 'B3', 'B4', 'B5', 'B7','B4_1'],
        'bandsGLCM': ['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'B4_asm', 'B4_contrast',"B4_corr", "B4_var","B4_idm","B4_savg", "B4_svar","B4_sent","B4_ent","B4_dvar","B4_dent","B4_imcorr1", "B4_imcorr2","B4_maxcorr","B4_diss","B4_inertia","B4_shade","B4_prom"],
        'FCCbands': ['B4', 'B3', 'B2'],
        'Landsat': 'LANDSAT/LE07/C01/T1'
    }

  if (year >= 1984 and year <= 1998 or year >=2003 and year <=2012):
    return{
        'bands': ['B1', 'B2', 'B3', 'B4', 'B5', 'B7'],
        'bandsClassify': ['B1', 'B2', 'B3', 'B4', 'B5', 'B7','B4_1'],
        'bandsGLCM': ['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'B4_asm', 'B4_contrast',"B4_corr", "B4_var","B4_idm","B4_savg", "B4_svar","B4_sent","B4_ent","B4_dvar","B4_dent","B4_imcorr1", "B4_imcorr2","B4_maxcorr","B4_diss","B4_inertia","B4_shade","B4_prom"],
        'FCCbands': ['B4', 'B3', 'B2'],
        'TextureBand': ['B4'],
        'Landsat': 'LANDSAT/LT05/C01/T1'
    }
  
  if (year == 1983):
    return{
        'bands': ['B1', 'B2', 'B3', 'B4', 'B5', 'B7'],
        'bandsClassify': ['B1', 'B2', 'B3', 'B4', 'B5', 'B7','B4_1'],
        'bandsGLCM': ['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'B4_asm', 'B4_contrast',"B4_corr", "B4_var","B4_idm","B4_savg", "B4_svar","B4_sent","B4_ent","B4_dvar","B4_dent","B4_imcorr1", "B4_imcorr2","B4_maxcorr","B4_diss","B4_inertia","B4_shade","B4_prom"],
        'FCCbands': ['B4', 'B3', 'B2'],
        'TextureBand': ['B4'],
        'Landsat': 'LANDSAT/LT04/C01/T1'
    }
  
  if (year >= 1979 and year <= 1982):
    return {
        'bands': ['B4_median', 'B5_median', 'B6_median','B7_median','B6_median_1'],
        'bandsClassify': ['B4_median', 'B5_median', 'B6_median', 'B7_median'],
        'FCCbands': ['B6_median', 'B5_median', 'B4_median'],
        'TextureBand': ['B6_median'],
        'Landsat': 'LANDSAT/LM03/C01/T1'
    }

  if (year >= 1975 and year <= 1978):
    return {
        'bands': ['B4_median', 'B5_median', 'B6_median','B7_median','B6_median_1'],
        'bandsClassify': ['B4_median', 'B5_median', 'B6_median', 'B7_median'],
        'FCCbands': ['B6_median', 'B5_median', 'B4_median'],
        'TextureBand': ['B6_median'],
        'Landsat': 'LANDSAT/LM02/C01/T2'
    }

  if (year >= 1972 and year <= 1974):
    return {
        'bands': ['B4_median', 'B5_median', 'B6_median','B7_median','B6_median_1'],
        'bandsClassify': ['B4_median', 'B5_median', 'B6_median', 'B7_median'],
        'FCCbands': ['B6_median', 'B5_median', 'B4_median'],
        'TextureBand': ['B6_median'],
        'Landsat': 'LANDSAT/LM01/C01/T2'
    }

In [ ]:
def landsat_product(landsat_id):
  if(landsat_id == 'LANDSAT/LM01/C01/T2' or landsat_id == 'LANDSAT/LM02/C01/T2' or landsat_id == 'LANDSAT/LM03/C01/T1'):
    image = ee.ImageCollection(landsat_id).filterDate(str(gtYearBegin) + '-01-01',str(gtYearEnd) + '-12-31').filter(ROI[roi]['doyFilterLandsat']).filterMetadata('CLOUD_COVER_LAND', 'less_than', 50).map(cloudMask).reduce(ee.Reducer.median())
    return {
      'image': image,
      'image_int': image.toByte()       
    }
  else:
    image_col = ee.ImageCollection(landsat_id).filterDate(str(gtYearBegin) + '-01-01',str(gtYearEnd) + '-12-31').filter(ROI[roi]['doyFilterLandsat'])
    return {
      'image': ee.Algorithms.Landsat.simpleComposite(image_col, 50, cloud, 40, True),
      'image_int': ee.Algorithms.Landsat.simpleComposite(image_col, 50, cloud, 40, False)
  }

In [ ]:
def add_indices(params, product):
  
  if(params['Landsat']=='LANDSAT/LC08/C01/T1'):
    swir  = product['image'].select('B6')
    nir   = product['image'].select('B5')
    red   = product['image'].select('B4')
    green = product['image'].select('B3')

    ndvi  = nir.subtract(red).divide(nir.add(red))
    ndbi  = swir.subtract(nir).divide(swir.add(nir))
    bi    = ndbi.subtract(ndvi)
    ndwi  = green.subtract(swir).divide(green.add(swir))

    product['image'] = product['image'].addBands([ndvi, ndbi, bi, ndwi])
    product['image'] = product['image'].rename('B1','B2','B3','B4','B5','B6','B7','B8','B9','B10','B11','ndvi','ndbi','bi','ndwi')

    params['bandsClassify'].extend(['ndvi', 'ndbi', 'bi', 'ndwi'])


  if(params['Landsat'] == 'LANDSAT/LE07/C01/T1'):
    swir  = product['image'].select('B5')
    nir   = product['image'].select('B4')
    red   = product['image'].select('B3')
    green = product['image'].select('B2')

    ndvi  = nir.subtract(red).divide(nir.add(red))
    ndbi  = swir.subtract(nir).divide(swir.add(nir))
    bi    = ndbi.subtract(ndvi)
    ndwi  = green.subtract(swir).divide(green.add(swir))

    product['image'] = product['image'].addBands([ndvi, ndbi, bi, ndwi])
    product['image'] = product['image'].rename('B1','B2','B3','B4','B5','B6_VCID_1','B6_VCID_2','B7','B8','ndvi','ndbi','bi','ndwi')

    params['bandsClassify'].extend(['ndvi', 'ndbi', 'bi', 'ndwi'])
    

  if(params['Landsat'] == 'LANDSAT/LM04/C01/T1' or params['Landsat'] == 'LANDSAT/LT05/C01/T1'):
    swir  = product['image'].select('B5')
    nir   = product['image'].select('B4')
    red   = product['image'].select('B3')
    green = product['image'].select('B2')

    ndvi  = nir.subtract(red).divide(nir.add(red))
    ndbi  = swir.subtract(nir).divide(swir.add(nir))
    bi    = ndbi.subtract(ndvi)
    ndwi  = green.subtract(swir).divide(green.add(swir))

    product['image'] = product['image'].addBands([ndvi, ndbi, bi, ndwi])
    product['image'] = product['image'].rename('B1','B2','B3','B4','B5','B6','B7','ndvi','ndbi','bi','ndwi')

    params['bandsClassify'].extend(['ndvi', 'ndbi', 'bi', 'ndwi'])


  if(params['Landsat'] == 'LANDSAT/LM01/C01/T2' or params['Landsat'] == 'LANDSAT/LM02/C01/T2' or params['Landsat'] == 'LANDSAT/LM03/C01/T1'):
    swir = product['image'].select('B6_median')
    nir = product['image'].select('B6_median')
    red = product['image'].select('B5_median')

    ndvi = nir.subtract(red).divide(nir.add(red))

    product['image'] = product['image'].addBands(ndvi)
    product['image'] = product['image'].rename('B4_median','B5_median','B6_median','B7_median','BQA_median','ndvi')

    params['bandsClassify'].append('ndvi')
  

In [ ]:
import time
ts = time.time()
timestamp = int(ts)

ROI = [
       {'cityName': 'PHL-Laguna', 'LatMax': 14.5758304600000006, 'LatMin': 13.9641799899999999, 'LonMax': 121.6324386600000054, 'LonMin': 121.0053329499999961, 'yearBegin':2019, 'yearEnd':2019, 'doyFilter':ee.Filter.And(ee.Filter.greaterThanOrEquals('doy',  1), ee.Filter.lessThanOrEquals('doy',  366)), 'doyFilterLandsat': ee.Filter.dayOfYear(1,366)},
      #  {'cityName': 'PHL-Laguna-Wet', 'LatMax': 14.5758304600000006, 'LatMin': 13.9641799899999999, 'LonMax': 121.6324386600000054, 'LonMin': 121.0053329499999961, 'yearBegin':1972, 'yearEnd':1972, 'doyFilter':ee.Filter.And(ee.Filter.greaterThanOrEquals('doy',  153), ee.Filter.lessThanOrEquals('doy',  305)), 'doyFilterLandsat': ee.Filter.dayOfYear(153, 305)},
]

In [ ]:
cloud = 30
spatial_resolution = 30
GoogleDriveFolderLC = "LandCoverMapGen-LC-"+ str(timestamp)
GoogleDriveFolderAA = 'LandCoverMapGen-AA'
n_sample = 10000
n_trees = 100
distance = 370000

In [ ]:
AA = []
year= ''
roi= ''
kernel = ee.Kernel.gaussian(1)
out_ext = ''

In [ ]:
ref_point = ee.FeatureCollection('users/rhtkhati/gt-pt-2500')

In [ ]:
for roi in range(0, len(ROI), 1):
  city_name = ROI[roi]['cityName']
  year_begin = ROI[roi]['yearBegin']
  year_end = ROI[roi]['yearEnd']
  for year in range(year_begin, year_end+1, 1):
    nSampleClass = []
    out_ext = ee.Geometry.Rectangle([ ROI[roi]['LonMin'], ROI[roi]['LatMin'], ROI[roi]['LonMax'], ROI[roi]['LatMax']])
    gtBounds = ref_point.geometry().bounds().getInfo()["coordinates"]
    gtLonMin = gtBounds[0][0][0]
    gtLatMin = gtBounds[0][0][1]
    gtLonMax = gtBounds[0][2][0]
    gtLatMax = gtBounds[0][2][1]

    gtExt = ee.Geometry.Rectangle([gtLonMin, gtLatMin, gtLonMax, gtLatMax])

    landsat_params = select_landsat(year_begin)

    gtYearBegin = year_begin
    gtYearEnd = year_end

    filter_ref_point = ref_point.filter(ee.Filter.And(ee.Filter.greaterThanOrEquals('year', gtYearBegin), ee.Filter.lessThanOrEquals('year', gtYearEnd), ROI[roi]['doyFilter']))

    dist = filter_ref_point.map(add_distance)
    filter_dist = ee.Filter.lt('d', distance)
    points = dist.filter(ee.Filter.eq('class', 1)).filter(filter_dist).limit(n_sample,"d")
    nSampleClass.insert(0,points.size())

    for i in range(2, 5, 1):
      randomSampleClass = dist.filter(ee.Filter.eq('class', i)).filter(filter_dist).limit(n_sample,"d")
      nSampleClass.insert(i-1, randomSampleClass.size())
      points = points.merge(randomSampleClass)

    nRandomSample = points.size().getInfo()
    if(nRandomSample > 300):
      print(city_name + "," + str(year_begin)+","+str(nRandomSample))

    product = landsat_product(landsat_params['Landsat'])
    print(product['image'].getInfo())

    add_indices(landsat_params, product)
    print(product['image'].getInfo())
    

    gt_image = product['image'].clip(gtExt)
    gt_image_int = product['image_int'].clip(gtExt)
    gt_image_texture = gt_image.addBands(gt_image_int.select(landsat_params['TextureBand']).toInt16().entropy(kernel))

    out_image = product['image'].clip(out_ext)
    out_image_int = product['image_int'].clip(out_ext)
    out_image_texture = out_image.addBands(out_image_int.select(landsat_params['TextureBand']).entropy(kernel))

    alg = ee.Classifier.smileRandomForest(n_trees)
    points = points.randomColumn()
    trainInput = gt_image_texture.sampleRegions(points.limit(n_sample/2, 'random', True), ['class'], 1)

    classifier = alg.train(trainInput, 'class', landsat_params['bandsClassify'])

    classified = out_image_texture.select(landsat_params['bandsClassify']).classify(classifier).focal_mode()
    compositeImage = out_image.select(landsat_params['FCCbands'])

    trainAccuracy = classifier.confusionMatrix()
    testingInput = gt_image_texture.sampleRegions(points.limit(n_sample/2, 'random', False), ['class'], 1)

    AA.append(ee.Feature(out_ext, 
                         {
                            'cityName': city_name,
                            'year': str(year_begin) + '-' + str(year_end),
                            'ts': str(timestamp),
                            'OA': trainAccuracy.accuracy(),
                            'KA': trainAccuracy.kappa(),
                            'CA': trainAccuracy.consumersAccuracy(),
                            'PA': trainAccuracy.producersAccuracy(),
                            'n_sample': nSampleClass
                         }))
    lcTask = ee.batch.Export.image.toDrive(**{
        'image': classified,
        'description': 'LC-' + city_name + '_' + str(year_begin) + '-' + str(year_end) + '_' + str(timestamp),
        'folder': GoogleDriveFolderLC,
        'fileNamePrefix': 'LC-' + city_name + '_' + str(year_begin) + '-' + str(year_end) + '_' + str(timestamp),
        'scale': spatial_resolution,
        'region': out_ext
    })

    fccTask = ee.batch.Export.image.toDrive(**{
        'image': compositeImage,
        'description':    'FCC-' + city_name + '-' + str(year_begin) + '-' + str(year_end) + '-' + str(timestamp),
        'folder': GoogleDriveFolderLC,
        'fileNamePrefix': 'FCC-' + city_name + '-' + str(year_begin) + '-' + str(year_end) + '-' + str(timestamp),
        'scale': spatial_resolution,
        'region': out_ext
    })

    lcTask.start()
    fccTask.start()
    
aaTask = ee.batch.Export.table.toDrive(**{
    'collection': ee.FeatureCollection(AA),
    'description': 'AA-'+ '-' + str(timestamp),
    'folder': GoogleDriveFolderAA,
    'fileNamePrefix': 'AA-'+ str(timestamp),
})

aaTask.start()

In [ ]:
import folium

## For Landsat 4, 5, 7, 8
mapid = out_image_texture.getMapId({'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3})

## For landsat 1-3
# mapid = out_image_texture.getMapId({'bands': ['B6_median', 'B5_median', 'B4_median'], 'min': 0, 'max': 255})

map = folium.Map(location=[14.4, 121.3]) 
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='Out Image Texture',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

In [ ]:
# import pandas as pd

# conf_val = (trainAccuracy.getInfo())
# conf_df = pd.DataFrame(conf_val).T
# conf_df[5] = df.sum(axis=1)
# conf_df.loc[5]= df.sum(axis=0)

# conf_df